In [ ]:

## Importing Libraries

import pyspark
from pyspark.sql import SparkSession as ss
from pyspark.sql.functions import regexp_extract, regexp_replace,avg, udf
from pyspark.sql.functions import round as Round
from pyspark.sql.functions import col, when
from pyspark.sql.functions import to_date
from pyspark.sql.functions import lower
from pyspark.sql.types import IntegerType
import re
from statistics import mean


spark = ss.builder.appName('ADDO-EXAM').getOrCreate()
sc= spark.sparkContext


**Note:**
For this task I had uploaded the recipe files to my Azure storage and perform operations by reading them from the Azure blob storage, since in real time enviroment the data is being stored on the cloud storage so it make sense to do this way by storing files on cloud and then performing spark tasks.

In [ ]:
# Authentication to access the files on my Azure storage
storage_account_name = "sparkexamstorage"  
storage_account_access_key = "QmRcItJlh4AE8ba/9x8sQSs2D6DhsusEf2CtNIiZBQRCBP84mRJR8eprOFGjeNtn5xzqzpAZu1XS+AStyAzBWg=="

# Containers on Azure Storage
inp_container = "data" ## Container name where recipies are present 
inp_file_type = "json"

out_container = "output" ## Container name where final result file containg only the beef recipies
out_file_type = "csv"



In [ ]:
# Configuring for Spark Application
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

In [ ]:
input_loc = dbutils.fs.ls(f"wasbs://{inp_container}@{storage_account_name}.blob.core.windows.net/")

output_container_path = f"wasbs://{out_container}@{storage_account_name}.blob.core.windows.net/"

### TASK 1

In [ ]:

# Since all the column are in string format and need to be corected according to the requiremnet:
# > Time is given in ISO format need to extract time in minutes.
# > Dish per person serving needs to be extracted from recipeYield columns 
# > Filling/removing of Null record.
# > Removing extra spaces/punctuations from description and ingredients columns.
# 
# Therefore for sucessful task completion for that some functions are needed to be defined as:


###########################################################
#### Function to extract the Time per dish in minutes #####
###########################################################

def get_time(ISOtime_):
    hours = 0
    minutes = 0
    
    # extract hours, based on regular expression
    hours_match = re.search(r'(\d+)H', ISOtime_)
    if hours_match:
        hours = int(hours_match.group(1))
        
    # extract minutes, based on regular expression
    minutes_match = re.search(r'(\d+)M', ISOtime_)
    if minutes_match:
        minutes = int(minutes_match.group(1))
        
    # Calculate total minutes
    total_minutes = hours * 60 + minutes
    return total_minutes

###################################################
#### Function to extract the serving per dish #####
###################################################

def extract_serving_value(s):
    num_regex = re.compile(r'\d+')
    if len(s)==1 and isinstance(s,int):
        return (s)
    else:
        nums = [int(num) for num in num_regex.findall(s)]
        return (mean(nums)) if nums else (1)

  

In [ ]:

###########################################################
##### TASK 1 ##############################################
########################################################### 

def PrePocessing(df):


  df2 = df.select("datePublished","name","cookTime","ingredients","prepTime","recipeYield")

  ####################################
  ##### Operation on datePublished ###
  ####################################

  # Converting the date to Date format from string
  df2 = df2.withColumn("Date Published", to_date(df2["datePublished"]))


  ####################################
  ##### Operation on Name Column #####
  ####################################


  # Remove extra spaces, punctuations and lower casing from the 'name' column
  df2 = df2.withColumn('Dish_Name', regexp_replace('name', '[^\w\s]+', '').alias('Name'))
  df2 = df2.withColumn('Dish_Name', regexp_replace('name', '\s+', ' ').alias('Name'))
  df2 = df2.withColumn('Dish_Name', lower(df2['name']))


  ####################################
  ##### Operation on Ingredients #####
  ####################################


  # Remove extra spaces, punctuations and lower casing from the 'ingredients' column
  df2 = df2.withColumn('Ingridients', regexp_replace('ingredients', '[^\w\s]+', '').alias('Ingridients'))
  df2 = df2.withColumn('Ingridients', regexp_replace('ingredients', '\s+', ' ').alias('Ingridients'))
  df2 = df2.withColumn('Ingridients', lower(df2['ingredients']))


  ####################################
  ##### Operation on Cook Time #######
  ####################################

  # Getting cook time in minutes 
  cooktime_func = udf(get_time, IntegerType())

  # Applying the Function to cooktime column
  df2 = df2.withColumn('Cook_Time', cooktime_func(df2['cookTime']))

  # Replace null values with the mean of previous values
  mean_val_CT = df2.select(avg(col('Cook_Time'))).collect()[0][0]
  df2 = df2.fillna({'Cook_Time': mean_val_CT})

  # Round the values of newly added cook and prep columns
  df2 = df2.withColumn('Cook_Time', Round(col('Cook_Time'), 0))


  ####################################
  ##### Operation on Prep Time #######
  ####################################

  # Getting Prep time in minutes 
  preptime_func = udf(get_time, IntegerType())

  # Applying the Function to preptime column
  df2 = df2.withColumn('Prep_Time', preptime_func(df2['prepTime']))

  # Replace null values with the mean of previous values
  mean_val_PT = df2.select(avg(col('Prep_Time'))).collect()[0][0]
  df2 = df2.fillna({'Prep_Time': mean_val_PT})

  # Round the values of newly added cook and prep columns
  df2 = df2.withColumn('Prep_Time', Round(col('Prep_Time'), 0))

  ####################################
  ##### Operation on recipeYield #####
  ####################################


  # Getting the Serving per dish 
  serving_func = udf(extract_serving_value, IntegerType())
  df2 = df2.withColumn('Serving', serving_func(df2['recipeYield']))

  # Round the values of newly added cook and prep columns
  df2 = df2.withColumn('Serving', Round(col('Serving'), 0))



  # Drop the old columns
  df2 = df2.drop('cookTime')
  df2 = df2.drop('prepTime')
  df2 = df2.drop('name')
  df2 = df2.drop('datePublished')
  df2 = df2.drop('recipeYield')
  df2 = df2.drop('ingredients')

  return df2






### TASK 2

In [ ]:

###########################################################
##### TASK 2 ##############################################
########################################################### 

def BeefRecipies(df2):


  # Create a new column by adding two existing columns
  df2 = df2.withColumn('Total_cook_time', col('Cook_Time') + col('Prep_Time'))

  # Create a new column based on the value of the 'sum' column
  df2 = df2.withColumn('Difficulty', when(col('Total_cook_time') < 30, 'Easy')
                                    .when((col('Total_cook_time') >= 30) & (col('Total_cook_time') <= 60), 'Medium')
                                    .when(col('Total_cook_time') > 60, 'Hard'))



  # Creating a table EXAM from dataframe, for quering the beef from record
  df2.createOrReplaceTempView('RECIPE')

  # SQL query that shows the dishes having beef in their ingredites or in dish name 
  beef_recipe = spark.sql("select * from RECIPE where (Ingridients like '%beef%') or (Dish_Name like '%beef%')")

  return beef_recipe



  

In [ ]:
# Creating dict of dataframes based on recipies
df = {}

# Iterate over all the saved records on the cloud storage and perform operations based on our tasks requirement.
for l in range(len(input_loc)):
  file_location = input_loc[l][0]
  df[file_location[-16:-5]] = spark.read.format(inp_file_type).load(file_location)
  output_folder = f"wasbs://{out_container}@{storage_account_name}.blob.core.windows.net/Output-{file_location[-16:-5]}/"

  # ##### TASK 1 
  df2 = PrePocessing(df[file_location[-16:-5]]) 

  # Persist the dataframe for future processing
  df[file_location[-16:-5]] = df2.persist()           ## df[file_location[-16:-5]] is dataframe of each recipe e.g: df['recipe-000'] contains
                                                      ## preprossed record of recipe-000 file and so on 

  # ##### TASK 2
  df2 = BeefRecipies(df[file_location[-16:-5]])


  ##### Writing the final output to the Azure cloud storage  
  (df2.coalesce(1).write.mode("overwrite").option("header", "true").format(out_file_type).save(output_folder))

  # Get the name of the output recipe CSV file that was just saved to Azure blob storage.
  files = dbutils.fs.ls(output_folder)
  output_file = [x for x in files if x.name.startswith("part-")]

  # Renameing the file name since the save file has a very large name
  dbutils.fs.mv(output_file[0].path,f"{output_container_path}/Output-{file_location[-16:-5]}/Final-Recipee-{file_location[-8:-5]}.csv")

  #break

# df2.show(5)




### CREATING DASHBOARD

In [ ]:
finalDF = BeefRecipies(df['recipes-000'].union(df['recipes-001']).union(df['recipes-002']))


In [ ]:
display(finalDF)

Date Published,Dish_Name,Ingridients,Cook_Time,Prep_Time,Serving,Total_cook_time,Difficulty
2010-11-23,french onion soup stuffed mushrooms,"2 tablespoons butter 2 whole large onions, halved and sliced thin 1/4 cup beef broth 7 dashes worcestershire sauce splash of red or white wine 1/2 cup grated gruyere cheese (can use swiss) kosher salt 24 whole white or crimini mushrooms, washed and stems removed minced parsley",30,20,8,50,Medium
2012-11-26,baked ziti,"2 tablespoons olive oil 3 cloves garlic, minced 1 whole large onion, diced 1 pound italian sausage 1 pound ground beef 1 can (28 ounce can) whole tomatoes, with juice 2 cans (14.5 ounce) tomato sauce or marinara sauce 2 teaspoons italian seasoning 1/2 teaspoon red pepper flakes salt and pepper, to taste 16 ounces, weight ziti or mostaciolli, cooked until not quite al dente 1 tub (15 ounce) whole milk ricotta cheese 1-1/2 pound mozzarella cheese, grated 1/2 cup grated parmesan cheese 1 whole egg fresh minced parsley",45,15,12,60,Medium
2013-01-14,sunday night stew,"stew 3 tablespoons olive oil 1 tablespoon butter 2 pounds beef stew meat (chuck roast cut into chunks) salt and pepper 1 whole medium onion, diced 3 cloves garlic, minced 4 ounces, weight tomato paste 4 cups low sodium beef stock or broth, more if needed for thinning several dashes worcestershire 1/2 teaspoon sugar 4 whole carrots, peeled and diced 2 whole turnips, peeled and diced 2 tablespoons minced fresh parsley mashed potatoes 5 pounds russet potatoes (peeled) 1 package (8 ounce) cream cheese, softened 1 stick butter, softened 1/2 cup heavy cream 1 teaspoon seasoned salt salt and pepper, to taste",180,15,8,195,Hard
2010-01-18,italian meatball soup,"meatballs: 3/4 pounds ground beef 1/2 cup freshly grated parmesan cheese 3 tablespoons fresh parsley, minced 1 whole egg 2 cloves garlic 1/2 teaspoon salt 1/2 teaspoon black pepper 1/4 teaspoon ground oregano 2 teaspoons lemon juice soup 3 tablespoons olive oil 7 cups low sodium beef stock 2 cups water 1/2 teaspoon salt 2 tablespoons tomato paste 3/4 cups onion, chopped 3/4 cups carrots, chopped 3/4 cups celery, chopped 1 cup russet potato, chopped (do not peel) 1/2 pound cabbage chopped grated parmesan cheese to serve tied in a cheesecloth bundle 4 tablespoons fresh parsley, minced 2 whole bay leaves 1 teaspoon peppercorns",45,30,8,75,Hard
2010-02-03,"drip beef, two ways","1 whole beef chuck roast, 2.5 to 4 pounds 1 can beef consomme or beef broth 3 tablespoons (heaping) italian seasoning 1 teaspoon salt 1/4 cup water 1/2 jar (16 oz) pepperoncini peppers, with juice buttered, toasted deli rolls",360,5,10,365,Hard
2010-02-15,spaghetti & meatballs,"meatballs: 3/4 pounds ground beef 3/4 pounds ground pork 3 cloves garlic, minced 3/4 cups fine bread crumbs 2 whole eggs 3/4 cups freshly grated parmesan 1/4 cup flat-leaf parsley, minced 1/4 teaspoon salt freshly ground black pepper splash of milk 1/2 cup olive oil sauce: 1 whole yellow onion, diced 3 cloves garlic, minced 1 whole (28-ounce) can whole tomatoes 1 whole 28 ounce can crushed tomatoes 1/2 cup white or red wine (optional) 1/4 teaspoon salt 1 teaspoon sugar freshly ground black pepper 1/4 cup flat-leaf parsley, minced 8 whole fresh basil leaves, chiffonade (optional) 2 pounds spaghetti, cooked to al dente",60,25,8,85,Hard
2010-03-30,passover brisket,"1 whole beef brisket, trimmed of all fat (5 to 8 pounds) 1 bottle (24 ounce) ketchup or chili sauce (make sure it's kosher) 1 package onion soup mix (make sure it's kosher)",420,10,8,430,Hard
2010-04-12,sloppy joes,"2 tablespoons butter 2-1/2 pounds ground beef 1/2 whole large onion, diced 1 whole large green bell pepper, diced 5 cloves garlic, minced 1-1/2 cup ketchup 1 cup water 2 tablespoons brown sugar 2 teaspoons chili powder (more to taste) 1 teaspoon dry mustard 1/2 teaspoon red pepper flakes (more to taste) worcestershire sauce, to taste 2 tablespoons tomato paste (optional) tabasco sauce (optional; to taste) salt to taste freshly ground black pepper, to tas

Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks